In [1]:
import json
import re
from itertools import groupby 
from string import punctuation
import random
import os
from glob import glob
from tqdm import tqdm
import pandas as pd

In [2]:
input_file_path = 'output/0-news.csv'
output_dir = 'output'

In [3]:
VALID_BANGLA_REGEX = re.compile(r'[^\u0980-\u09FF ।,?!\(\)-.”“/;"‘:\']+')
SPLIT_REGEX = re.compile(r'\n|।|\?')
SPLIT_BY_NEWLINE_REGEX = re.compile(r'\n')
EMOJI_REGEX = re.compile(
    "(["
    "\U0001F1E0-\U0001F1FF"  # flags (iOS)
    "\U0001F300-\U0001F5FF"  # symbols & pictographs
    "\U0001F600-\U0001F64F"  # emoticons
    "\U0001F680-\U0001F6FF"  # transport & map symbols
    "\U0001F700-\U0001F77F"  # alchemical symbols
    "\U0001F780-\U0001F7FF"  # Geometric Shapes Extended
    "\U0001F800-\U0001F8FF"  # Supplemental Arrows-C
    "\U0001F900-\U0001F9FF"  # Supplemental Symbols and Pictographs
    "\U0001FA00-\U0001FA6F"  # Chess Symbols
    "\U0001FA70-\U0001FAFF"  # Symbols and Pictographs Extended-A
    "\U00002702-\U000027B0"  # Dingbats
    "])"
    )


def is_all_bangla(text, allowed_threshold=96.9):
    """
    Allows some non-bangla characters (excluding punctuations) and 
    finds out if contains greater than <percent> characters are from Bangla language
    """
    valid_bangla_text = VALID_BANGLA_REGEX.sub('', text)
    valid_bangla_text_len = len(valid_bangla_text)
    all_text_len = len(text)
    
    percent_bangla = (valid_bangla_text_len*100.0) / all_text_len

    if percent_bangla < allowed_threshold:
        return False
    return True
    
    
    
def is_valid(text):
    l= len(text)
    if l<MIN_CHAR_LEN or l>MAX_CHAR_LEN:
        return False
    if len(text.split())<MIN_WORD_LEN:
        return False
    if not is_all_bangla(text):
        return False
    
    return True

def split_sentences(text):
    return SPLIT_REGEX.split(text)


def split_sentences_from_paragraphs(paragraphs):
    sentences = []
    for p in paragraphs:
        sentences.append(SPLIT_REGEX.split(p))
    return sentences


def split_paragraphs(text):
    return SPLIT_BY_NEWLINE_REGEX.split(text)


def remove_repeating_puncts(texts, punctuations=set(punctuation+ '।\n')):
    # ref: https://stackoverflow.com/a/32485876
    newtext = []
    for k, g in groupby(texts):
        if k in punctuations:
            newtext.append(k)
        else:
            newtext.extend(g)
    return ''.join(newtext)


def remove_emoticons(text):
    # Ref: https://gist.github.com/Alex-Just/e86110836f3f93fe7932290526529cd1#gistcomment-3208085
    # Ref: https://en.wikipedia.org/wiki/Unicode_block
    text = re.sub(EMOJI_REGEX, r' ', text)
    return text


def remove_non_bangla_characters(text):
    if isinstance(text, str):
        cleaned = VALID_BANGLA_REGEX.sub(' ', text)
        return cleaned
    elif isinstance(text, list):
        results = []
        for item in text:
            cleaned = VALID_BANGLA_REGEX.sub(' ', item)
            results.append(cleaned)
        return results
    else:
        raise Exception('invalid param')
        
    
def is_poem(paragraphs):
    """Find out if blog article is poem or not, by analyzing avg. paragraph length"""
    
    total_len = 0
    for p in paragraphs:
        if not isinstance(p, str):
            print('error ', p)
        total_len += len(p)
    avg_para_len = total_len / len(paragraphs)

    if avg_para_len < 100:
        return True
    else:
        return False

In [5]:
input_df = pd.read_csv(input_file_path)
input_df.head()

,URL,source,text
0,http://www.bd-pratidin.com/first-page/2020/01/...,news,"প্রধানমন্ত্রী শেখ হাসিনা বলেছেন, বাংলাদেশ একটি..."
1,https://dailysylhet.com/details/424386,news,কমলগঞ্জ প্রতিনিধি:: “আসুন বায়ুদূষণ রোধ করি” এই...
2,http://www.bd-pratidin.com/home/printnews/4831...,news,পররাষ্ট্রমন্ত্রীর দিল্লি ও স্বরাষ্ট্রমন্ত্রীর ...
3,https://dailysylhet.com/details/438228,news,কমলগঞ্জ প্রতিনিধি:: মৌলভীবাজারের কমলগঞ্জ উপজেল...
4,http://www.bd-pratidin.com/first-page/2019/10/...,news,প্রথম শ্রেণির ক্রিকেটে পারিশ্রমিক বাড়ানোসহ ১১ ...


In [58]:
all_chars = []
char_map = {}
count = 0
count_not_bangla = 0

filtered_texts = []
for idx, row in tqdm(input_df.iterrows()):
    text = row['text']
    url = row['URL']
    if not isinstance(text, str):
        continue

    text = remove_repeating_puncts(text)
    text = remove_emoticons(text)
    if not is_all_bangla(text):
        count_not_bangla += 1
        continue

    paragraphs = split_paragraphs(text)

    if is_poem(paragraphs):
        print("Found poem: ", url)
        count += 1
        continue
    
    filtered_texts.append((text, url))
    
print(count)
print(count_not_bangla)

  0%|          | 399/932438 [00:01<113:42:34,  2.28it/s]

Found poem:  http://www.protidinersangbad.com/international/178530/%E0%A6%B8%E0%A7%8C%E0%A6%A6%E0%A6%BF%E0%A6%B0-%E0%A6%86%E0%A6%AC%E0%A6%B9%E0%A6%BE-%E0%A6%AC%E0%A6%BF%E0%A6%AE%E0%A6%BE%E0%A6%A8%E0%A6%AC%E0%A6%A8%E0%A7%8D%E0%A6%A6%E0%A6%B0%E0%A7%87-%E0%A6%A1%E0%A7%8D%E0%A6%B0%E0%A7%8B%E0%A6%A8-%E0%A6%B9%E0%A6%BE%E0%A6%AE%E0%A6%B2%E0%A6%BE/print
Found poem:  https://dailysylhet.com/details/326140 
Found poem:  http://www.protidinersangbad.com/todays-newspaper/find-jobs/170334/পিএসসি-নন-ক্যাডারে-১-হাজার-৫৯৭-নিয়োগ


  0%|          | 702/932438 [00:01<55:47:20,  4.64it/s] 

Found poem:  http://www.bd-pratidin.com/home/printnews/468351/2019-10-23
Found poem:  http://www.bd-pratidin.com/entertainment-news/2019/07/18/440885
Found poem:  http://www.protidinersangbad.com/todays-newspaper/find-jobs/161200/%E0%A6%AC%E0%A6%BF%E0%A6%AD%E0%A6%BF%E0%A6%A8%E0%A7%8D%E0%A6%A8-%E0%A6%AA%E0%A7%8D%E0%A6%B0%E0%A6%A4%E0%A6%BF%E0%A6%B7%E0%A7%8D%E0%A6%A0%E0%A6%BE%E0%A6%A8%E0%A7%87-%E0%A6%A8%E0%A6%BF%E0%A7%9F%E0%A7%8B%E0%A6%97/print
Found poem:  http://www.protidinersangbad.com/todays-newspaper/find-jobs/162842/%E0%A6%87%E0%A6%B8%E0%A7%8D%E0%A6%9F-%E0%A6%93%E0%A7%9F%E0%A7%87%E0%A6%B8%E0%A7%8D%E0%A6%9F-%E0%A6%87%E0%A6%89%E0%A6%A8%E0%A6%BF%E0%A6%AD%E0%A6%BE%E0%A6%B0%E0%A7%8D%E0%A6%B8%E0%A6%BF%E0%A6%9F%E0%A6%BF%E0%A6%A4%E0%A7%87-%E0%A6%A8%E0%A6%BF%E0%A7%9F%E0%A7%8B%E0%A6%97/print
Found poem:  http://www.protidinersangbad.com/todays-newspaper/find-jobs/162846/%E0%A6%A8%E0%A6%BF%E0%A7%9F%E0%A7%8B%E0%A6%97-%E0%A6%A6%E0%A7%87%E0%A6%AC%E0%A7%87-%E0%A6%8F%E0%A6%A8%E0%A6%86%E0%A6%B0%E0%

  0%|          | 1147/932438 [00:02<19:14:52, 13.44it/s]

...

Found poem:  http://old.dailynayadiganta.com/detail/printcopy/180066
Found poem:  http://www.bd-pratidin.com/coronavirus/2020/02/23/504786
Found poem:  http://old.dailynayadiganta.com/detail/news/120055
Found poem:  http://old.dailynayadiganta.com/detail/news/119797
Found poem:  http://old.dailynayadiganta.com/detail/printcopy/134125
Found poem:  http://old.dailynayadiganta.com/detail/printcopy/131912
Found poem:  http://www.kalerkantho.com/home/printnews/791550/2019-07-15
Found poem:  http://www.dailynayadiganta.com/syllabus /454611/-ইংরেজি-১২-নম্বর-প্রশ্ন-জবধৎৎধহমব
Found poem:  http://www.dailynayadiganta.com/news/printarticle/455266
Found poem:  http://www.dailynayadiganta.com/daily/458843/জা-না-অজানা-তরল-সোনা
Found poem:  http://www.dailynayadiganta.com/news/printarticle/455558
Found poem:  http://old.dailynayadiganta.com/detail/printcopy/268892
Found poem:  http://old.dailynayadiganta.com/detail/printcopy/139825
Found poem:  http://old.dailynayadiganta.com/detail/printcopy/139819


100%|█████████▉| 932102/932438 [10:25<00:00, 1573.04it/s]

Found poem:  http://www.bd-pratidin.com/home/printnews/106369/2015-10-29
Found poem:  http://www.kalerkantho.com/home/printnews/848280/2019-12-08
Found poem:  http://www.bd-pratidin.com/home/printnews/476951/2019-11-22
Found poem:  http://www.bd-pratidin.com/home/printnews/166899/2016-08-31
Found poem:  http://www.bd-pratidin.com/home/printnews/299760/2018-01-22
Found poem:  http://www.bd-pratidin.com/home/printnews/299470/2018-01-21
Found poem:  http://dailysangram.info/post/378519-নারায়ণগঞ্জে-মশার-কয়েল-থেকে-আগুন-একই-পরিবারের-দগ্ধ-৪
Found poem:  http://www.dailynayadiganta.com/news/printarticle/478705
Found poem:  http://www.dailynayadiganta.com/syllabus /453614/ইংরেজি-১২-নম্বর-প্রশ্ন-জবধৎৎধহমব
Found poem:  http://www.dailynayadiganta.com/syllabus /453154/-ইংরেজি-১২-নম্বর-প্রশ্ন-জবধৎৎধহমব
Found poem:  http://www.dailynayadiganta.com/syllabus /453329/এসএসসি-পরীক্ষার-প্রস্তুতি-সংখ্যা-২-রসায়ন-নবম-অধ্যায়-এসিড--ক্ষারক-সমতা-
Found poem:  http://www.dailynayadiganta.com/syllabus /453611/এসএস

100%|██████████| 932438/932438 [10:25<00:00, 1490.92it/s]

Found poem:  http://www.bd-pratidin.com/home/printnews/323783/2018-04-20
Found poem:  http://www.bd-pratidin.com/home/printnews/116144/2015-12-20
Found poem:  https://www.dailynayadiganta.com/daily/477563/দুই-গোয়েন্দার-অভিযান-
Found poem:  http://www.dailynayadiganta.com/news/printarticle/453613
Found poem:  http://www.dailynayadiganta.com/news/printarticle/453612
Found poem:  http://old.dailynayadiganta.com/detail/printcopy/181507
Found poem:  http://www.bd-pratidin.com/home/printnews/502508/2020-02-16
Found poem:  http://www.kalerkantho.com/online/lifestyle/2015/11/29/296146
Found poem:  http://old.dailynayadiganta.com/detail/printcopy/29880
Found poem:  http://old.dailynayadiganta.com/detail/news/5796
19867
12059


# Extract documents

In [59]:
paragraphs_for_next_step = []
documents = []
need_document = 18000

rejected_count = 0
for text, url in tqdm(filtered_texts):
    # if already found n documents needed, break
    if len(documents) >= need_document:
        break
        
    if not isinstance(text, str):
        continue
        
    paragraphs = split_paragraphs(text)
    cleaned_paragraphs = remove_non_bangla_characters(paragraphs)
    string = " ".join(cleaned_paragraphs)
    
    if len(paragraphs) < 3:
        rejected_count += 1
        continue
        
    if len(string) < 800: # character count
        rejected_count += 1
        continue
    
    # trim documents with more than 10 paragraphs 
    if len(paragraphs) > 10:
        no_of_paragraphs = random.randint(3, 10)
        cleaned_paragraphs = cleaned_paragraphs[:no_of_paragraphs]
        cleaned_text = '\n'.join(cleaned_paragraphs)
        
        words = cleaned_text.split()
        documents.append({'text': cleaned_text, 'url':url, 'para_count': no_of_paragraphs, 'word_count': len(words)})
        
        for p in paragraphs[:no_of_paragraphs]:
            paragraphs_for_next_step.append((p, url))
    else:
        cleaned_text = '\n'.join(cleaned_paragraphs)
        words = cleaned_text.split()
        documents.append({'text': cleaned_text, 'url':url, 'para_count': len(paragraphs), 'word_count': len(words)})
        

print('Rejected documents: ', rejected_count)
print("Got clean documents: ", len(documents))
print("Left paragraph for next: ", len(paragraphs_for_next_step))

  2%|▏         | 21839/900512 [00:01<00:55, 15713.78it/s]

Rejected documents:  5453
Got clean documents:  18000
Left paragraph for next:  22490


# Check stats and export Document CSV


In [60]:
document_df = pd.DataFrame(documents)
document_df.describe()

,para_count,word_count
count,18000.000000,18000.000000
mean,6.041111,290.700111
std,2.076855,197.512444
min,3.000000,7.000000
25%,4.000000,171.000000
50%,6.000000,235.000000
75%,8.000000,342.000000
max,10.000000,2306.000000


In [61]:
# filter lower word count documents, if some slipped through
document_df = document_df[document_df['word_count'] > 40]

# drop some columns to match with our standard CSV format
document_df = document_df.drop('para_count', axis=1)
document_df = document_df.drop('word_count', axis=1)
document_df['source'] = 'newspaper'

In [62]:
document_df.to_csv(os.path.join('output', '1-newspaper-documents.csv'), index=None)

# Extract Paragraphs

In [63]:
sentences_for_next_step = []
paragraphs = []

rejected = 0
for text, url in tqdm(paragraphs_for_next_step):
    if not isinstance(text, str):
        continue
        
    sentences = split_sentences(text)
    sentences = remove_non_bangla_characters(sentences)
    
    if len(text) < 200:
        for s in sentences:
            sentences_for_next_step.append((s, url))
        rejected += 1
        continue
        
    if len(sentences) < 3 or len(sentences) > 15:
        for s in sentences:
            sentences_for_next_step.append((s, url))
        rejected += 1
        continue    
    
    cleaned_text = "। ".join(sentences)
    word_count = len(cleaned_text.split())
    paragraphs.append({'text': cleaned_text, 'url': url, 'word_count': word_count, 'sent_count': len(sentences) })
    
print('Rejected paragraphs: ', rejected)

100%|██████████| 22490/22490 [00:00<00:00, 64370.16it/s]

Rejected paragraphs:  10495


# Check stats and export Paragraph CSV

In [64]:
# check stats
para_df = pd.DataFrame(paragraphs)
para_df.describe()

,sent_count,word_count
count,11995.000000,11995.000000
mean,6.260275,69.205419
std,2.873804,36.770215
min,3.000000,24.000000
25%,4.000000,43.000000
50%,5.000000,57.000000
75%,8.000000,85.000000
max,15.000000,375.000000


In [65]:
# filter lower word count documents, if some slipped through
para_df = para_df[para_df['word_count'] > 20]

# drop some columns to match with our standard CSV format
para_df = para_df.drop('sent_count', axis=1)
para_df = para_df.drop('word_count', axis=1)
para_df['source'] = 'newspaper'

para_df.to_csv(os.path.join(output_dir, '1-newspaper-paragraphs.csv'), index=None)

# Extract sentences

In [66]:
rejected = 0
sentences = []
for s, url in tqdm(sentences_for_next_step):
    if not isinstance(s, str):
        continue
    s = remove_non_bangla_characters(s)
    if len(s) < 20:
        rejected += 1
        continue
    
    words = s.split()
    if len(words) < 3 and len(words) > 15:
        rejected += 1
        continue
    
    sentences.append({'text': s, 'word_count': len(words), 'char_count': len(s)})
    
print("Rejected sentences: ", rejected)

100%|██████████| 32634/32634 [00:00<00:00, 316961.37it/s]

Rejected sentences:  8964


# Check stats and export sentenc CSVs¶

In [67]:
sentence_df = pd.DataFrame(sentences)
sentence_df.describe()

,char_count,word_count
count,23670.000000,23670.000000
mean,76.271990,11.999282
std,57.848331,8.592042
min,20.000000,2.000000
25%,41.000000,7.000000
50%,62.000000,10.000000
75%,93.000000,15.000000
max,1809.000000,266.000000


In [68]:
# filter lower word count sentences, if some slipped through
sentence_df = sentence_df[sentence_df['word_count'] > 3]

# drop some columns to match with our standard CSV format
sentence_df = sentence_df.drop('char_count', axis=1)
sentence_df = sentence_df.drop('word_count', axis=1)
sentence_df['source'] = 'newspaper'

sentence_df.to_csv(os.path.join(output_dir, '1-newspaper-sentences.csv'), index=None)